# Rodent Brain Atlas Registration and Electrode Deformation

This notebook demonstrates a complete pipeline for registering rodent brain atlases and performing electrode deformation analysis. The workflow includes:

1. **Initial Setup and Data Loading**
2. **Rigid Registration** - Center alignment of atlas to subject
3. **Affine Registration** - Linear alignment refinement  
4. **Non-linear Registration** - Deformable registration for precise anatomical matching
5. **Electrode Deformation Analysis** - Modeling tissue deformation around implanted electrodes

## Overview

This registration pipeline uses the Waxholm Space (WHS) rat brain atlas to register with individual subject brain images, followed by specialized electrode deformation modeling. The process is essential for:

- Accurate anatomical labeling of subject brain regions
- Understanding tissue displacement due to electrode implantation
- Quantifying registration quality through Jacobian determinant analysis

## Dependencies

The notebook requires the following packages:
- `nilearn` - Neuroimaging analysis and visualization
- `nibabel` - NIfTI image I/O
- `SimpleITK` - Image registration toolkit
- `MONAI` - Medical imaging transformations
- `numpy` - Numerical computing

---

## 1. Import Libraries and Initialize Environment

Import all necessary libraries for image processing, registration, and visualization.

In [ ]:

import os
import sys
# All dependencies are now in the current directory
import nilearn.image as ni
import nibabel as nb
from nilearn.plotting import plot_anat, plot_prob_atlas, show, plot_stat_map
import SimpleITK as sitk
from utils import pad_nifti_image, multires_registration, interpolate_zeros
from aligner import Aligner
from warp_utils import apply_warp
import numpy as np
from monai.transforms import LoadImage, EnsureChannelFirst
from warper import Warper



# %matplotlib notebook
# import gui


## 2. Configure File Paths

Define all input and output file paths for the registration pipeline. 

**Input Files:**
- Subject BSE T2 image
- Atlas BSE T2 template  
- Atlas anatomical labels

**Output Files:**
- Registered atlas images at each stage
- Transformation files
- Quality metrics (Jacobian determinants)

In [ ]:
# load original reoriented image and resample to 1mm cubic voxels

fname = "/deneb_disk/RodentTools/for_Seymour/11_15_2025/MRI/Raw T2/r57/r57_A06_RARE_T2_3D_WholeBrain_20250724092509_80001.reoriented.nii.gz"
subbase="/deneb_disk/RodentTools/for_Seymour/11_15_2025/MRI/Raw T2/r57/R57"

re_image = ni.load(fname)
resampled_re_image = ni.resample_img(re_image, target_affine=np.eye(3)*1.0, interpolation='linear')
ni.save(resampled_re_image, subbase+".reoriented.nii.gz")


#subbase = "/deneb_disk/RodentTools/data/test4/29408.native"#

sub_bse_t2 = "/deneb_disk/RodentTools/for_Seymour/11_15_2025/MRI/Raw T2/r57/R57.reoriented.bse.nii.gz" #subbase+".bfc.nii.gz"

atlas_bse_t2 = "/deneb_disk/RodentTools/Atlases/Waxholm/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_T2star_v1.01.bse.nii.gz"
atlas_labels = "/deneb_disk/RodentTools/Atlases/Waxholm/WHS_SD_rat_atlas_v4_pack/WHS_SD_rat_atlas_v4.nii.gz"

centered_atlas = subbase+".atlas.cent.nii.gz"
centered_atlas_labels = subbase+".atlas.cent.label.nii.gz"
cent_transform_file = subbase+".cent.reg.tfm"
inv_cent_transform_file = subbase+".cent.reg.inv.tfm"
centered_atlas_linreg = subbase+".atlas.lin.nii.gz"
centered_atlas_linreg_labels = subbase+".atlas.lin.label.nii.gz"
lin_reg_map_file = subbase+".lin_ddf.map.nii.gz"


In [ ]:

nonlin_reg_map_file = subbase+".nonlin_ddf.map.nii.gz"
inv_nonlin_reg_map_file = subbase+".inv.nonlin_ddf.map.nii.gz"
centered_atlas_nonlinreg = subbase+".atlas.nonlin.nii.gz"
centered_atlas_nonlinreg_labels = subbase+".atlas.nonlin.label.nii.gz"
jac_det_file = subbase+".jacobian_det.nii.gz"
inv_jac_det_file = subbase+".inv.jacobian_det.nii.gz"

## 3. Initial Data Visualization

Visualize the input data before registration to understand the initial alignment between subject and atlas images.

In [ ]:

plot_anat(sub_bse_t2)
d = plot_anat(atlas_bse_t2)
d.add_contours(atlas_labels, cmap="prism")


In [ ]:
d=plot_anat(sub_bse_t2, vmax=np.percentile(nb.load(sub_bse_t2).get_fdata(), 99.9),vmin=0)
d.add_contours(atlas_labels, cmap="prism")


## 4. Rigid Registration (Center Alignment)

Perform initial rigid registration to align the center of mass between the atlas and subject images. This step corrects for gross positioning differences and provides a good starting point for subsequent registrations.

**Key Steps:**
1. Initialize centered transform using geometry
2. Perform multi-resolution registration
3. Save forward and inverse transformations
4. Apply transformation to atlas and labels

In [ ]:
fixed_image = sitk.ReadImage(sub_bse_t2, sitk.sitkFloat32)
moving_image = sitk.ReadImage(atlas_bse_t2, sitk.sitkFloat32)
initial_transform = sitk.CenteredTransformInitializer(
    fixed_image,
    moving_image,
    sitk.Euler3DTransform(),
    sitk.CenteredTransformInitializerFilter.GEOMETRY,
)

final_transform, _ = multires_registration(
    fixed_image, moving_image, initial_transform)


# save the transformation in a file
sitk.WriteTransform(final_transform, cent_transform_file)

# invert the transform and also save to a file
inv_transform = final_transform.GetInverse()
sitk.WriteTransform(inv_transform, inv_cent_transform_file)

# load from the file and apply the transformation
final_transform = sitk.ReadTransform(cent_transform_file)
moved_image = sitk.Resample(moving_image, fixed_image, final_transform)

sitk.WriteImage(moved_image, centered_atlas)

moving_image = sitk.ReadImage(atlas_labels, sitk.sitkUInt16)
moved_image = sitk.Resample(
    moving_image,
    fixed_image,
    transform=final_transform,
    interpolator=sitk.sitkNearestNeighbor,
)
sitk.WriteImage(moved_image, centered_atlas_labels)




### 4.1 Visualize Rigid Registration Results

Check the quality of the rigid registration by overlaying the centered atlas with the subject image.

In [ ]:

plot_anat(centered_atlas)
d=plot_anat(sub_bse_t2, vmax=np.percentile(nb.load(sub_bse_t2).get_fdata(), 99.9),vmin=0)
d.add_contours(centered_atlas_labels, cmap="prism")

## 5. Affine Registration

Perform affine (linear) registration to account for scaling, rotation, and shearing differences between the atlas and subject. This builds upon the rigid registration to provide better anatomical alignment.

**Parameters:**
- **Loss function:** Cross-correlation (CC)
- **Transformation type:** Affine (12 degrees of freedom)

In [ ]:
aligner = Aligner()
aligner.affine_reg(
    fixed_file=sub_bse_t2,
    moving_file=centered_atlas,
    output_file=centered_atlas_linreg,
    ddf_file=lin_reg_map_file,
    loss="cc",
)


### 5.1 Apply Affine Transform to Labels

Apply the computed affine transformation to the atlas labels using the displacement field.

In [ ]:
d=plot_anat(sub_bse_t2, vmax=np.percentile(nb.load(sub_bse_t2).get_fdata(), 99.9),vmin=0)
d.add_contours(centered_atlas_linreg, cmap="prism")

In [ ]:
disp_field, meta = LoadImage(image_only=False)(lin_reg_map_file)
disp_field = EnsureChannelFirst()(disp_field)
print(disp_field.shape)

at1, meta = LoadImage(image_only=False)(centered_atlas_labels)
at_lab = EnsureChannelFirst()(at1)
print(at_lab.shape)

warped_lab = apply_warp(
    disp_field[None,], at_lab[None,], at_lab[None,], interp_mode="nearest"
)
nb.save(
    nb.Nifti1Image(warped_lab[0, 0].detach().cpu().numpy(), at_lab.affine),
    centered_atlas_linreg_labels,
)


### 5.2 Compare Registration Results

Visual comparison between rigid and affine registration results to assess improvement in anatomical alignment.

In [ ]:
d = plot_anat(sub_bse_t2, vmax=np.percentile(nb.load(sub_bse_t2).get_fdata(), 99),vmin=0)
d.add_contours(centered_atlas_labels, cmap="prism")
d = plot_anat(sub_bse_t2, vmax=np.percentile(nb.load(sub_bse_t2).get_fdata(), 99.95),vmin=np.percentile(nb.load(sub_bse_t2).get_fdata(), 15))
d.add_contours(centered_atlas_linreg_labels, cmap="hsv")


## 6. Non-linear Registration

Perform deformable (non-linear) registration to capture local anatomical variations that cannot be corrected by linear transformations alone. This step uses a neural network-based approach for precise anatomical matching.

**Parameters:**
- **Network input size:** 64×64×64
- **Learning rate:** 1e-4
- **Maximum epochs:** 5000
- **Loss function:** Cross-correlation (CC)
- **Regularization penalty:** 1.0

In [ ]:
nonlin_reg = Warper()
nonlin_reg.nonlinear_reg(
    target_file=sub_bse_t2,
    moving_file=centered_atlas_linreg,
    output_file=centered_atlas_nonlinreg,
    ddf_file=nonlin_reg_map_file,
    inv_ddf_file=inv_nonlin_reg_map_file,
    reg_penalty=1,  # originally it was 1
    nn_input_size=64,
    lr=1e-4,
    max_epochs=5000,
    loss="cc",
    jacobian_determinant_file=jac_det_file,
    inv_jacobian_determinant_file=inv_jac_det_file,
)


### 6.1 Apply Non-linear Transformation

Apply the computed deformation field to transform the atlas labels to match the subject anatomy.

In [ ]:
disp_field, meta = LoadImage(image_only=False)(nonlin_reg_map_file)
disp_field = EnsureChannelFirst()(disp_field)
print(disp_field.shape)

at1, meta = LoadImage(image_only=False)(centered_atlas_linreg_labels)
at_lab = EnsureChannelFirst()(at1)
print(at_lab.shape)

warped_lab = apply_warp(
    disp_field[None,], at_lab[None,], at_lab[None,], interp_mode="nearest"
)
nb.save(
    nb.Nifti1Image(
        np.uint16(warped_lab[0, 0].detach().cpu().numpy()), at_lab.affine),
    centered_atlas_nonlinreg_labels,
)


### 6.2 Compare Registration Stages

Visual comparison between affine and non-linear registration results to evaluate the improvement in anatomical detail matching.

In [ ]:
d = plot_anat(sub_bse_t2, vmax=np.percentile(nb.load(sub_bse_t2).get_fdata(), 99),vmin=0)
d.add_contours(centered_atlas_linreg_labels, cmap="prism")
d = plot_anat(sub_bse_t2, vmax=np.percentile(nb.load(sub_bse_t2).get_fdata(), 99),vmin=0)
d.add_contours(centered_atlas_nonlinreg_labels, cmap="prism")


## 7. Registration Quality Assessment

Evaluate the quality of the non-linear registration using Jacobian determinant analysis. The Jacobian determinant measures local volume changes and helps identify areas of expansion/compression in the deformation field.

**Interpretation:**
- **Jacobian = 1:** No volume change (perfect preservation)
- **Jacobian > 1:** Local expansion 
- **Jacobian < 1:** Local compression
- **Values close to 0:** Potential registration artifacts

In [ ]:
plot_anat(jac_det_file, colorbar=True)

jac=nb.load(jac_det_file)
jac = jac.get_fdata() - 1

from nilearn.image import new_img_like
jac = new_img_like(sub_bse_t2,jac)
plot_stat_map(jac,sub_bse_t2,title='Jac det')

### 7.1 Forward Jacobian Determinant

Analysis of the forward transformation (atlas → subject) Jacobian determinant.

In [ ]:
plot_anat(inv_jac_det_file, colorbar=True)

jac=nb.load(inv_jac_det_file)
jac = jac.get_fdata() - 1

from nilearn.image import new_img_like
jac = new_img_like(sub_bse_t2,jac)
plot_stat_map(jac,sub_bse_t2,title='Jac det inv')


### 7.2 Inverse Jacobian Determinant  

Analysis of the inverse transformation (subject → atlas) Jacobian determinant to assess bidirectional registration quality.

## 8. Electrode Deformation Analysis

This section demonstrates specialized electrode deformation modeling, which is crucial for understanding how brain tissue deforms around implanted electrodes. This analysis helps in:

- **Correcting registration artifacts** caused by electrode implantation
- **Modeling tissue displacement** around foreign objects
- **Improving anatomical accuracy** in regions affected by electrodes

The deformation modeling uses specific target points that represent the electrode center and tip positions, allowing for precise modeling of the cylindrical deformation field around the electrode tract.

### Key Parameters:
- **Target points:** Coordinates defining electrode center and tip
- **Deformation model:** Cylindrical field around electrode tract  
- **Label handling:** Nearest-neighbor interpolation for discrete labels

In [ ]:
# Import required modules for electrode deformation
from deform_image_by_electrode import deform_image_by_electrode
import SimpleITK as sitk

### 8.1 First Electrode Deformation

Model deformation around the first electrode using the linearly registered atlas as the starting point.

In [ ]:
# First electrode deformation example
# Define paths for the first electrode
target_path = subbase + ".atlas.lin.label.nii.gz"
target_electrode_path = subbase + ".atlas.lin.electrode1.label.nii.gz"
target_electrode_deformed_path = subbase + ".atlas.lin.electrode1.deformed.label.nii.gz"
target_electrode_deformed_path = subbase + ".reoriented.atlas.lin.electrode1.deformed.label.nii.gz"

# Define target points for first electrode (center and tip)
target_pts = [[-2.08, -1.74, 6.86], [-2.18, -3.12, -0.54]]

# Perform electrode deformation
deform_image_by_electrode(
    target_path=target_path,
    target_electrode_path=target_electrode_path,
    target_electrode_deformed_path=target_electrode_deformed_path,
    target_pts=target_pts, 
    islabel=True  # mark center of electrode and tip of electrode
)

### 8.2 Second Electrode Deformation

Model deformation around the second electrode using the result from the first electrode deformation. This sequential approach accounts for the cumulative effects of multiple electrode insertions.

In [ ]:
# Second electrode deformation example
# Define paths for the second electrode (using first electrode's deformed result as input)
target_path = subbase + ".reoriented.atlas.lin.electrode1.deformed.label.nii.gz"
target_electrode_path = subbase + ".reoriented.atlas.lin.electrode2.label.nii.gz"
target_electrode_deformed_path = subbase + ".reoriented.atlas.lin.electrode2.deformed.label.nii.gz"

# Define target points for second electrode (center and tip)
target_pts = [[-4.00, -5.72, 6.86], [-4.70, -6.62, -0.40]]

# Perform electrode deformation
deform_image_by_electrode(
    target_path=target_path,
    target_electrode_path=target_electrode_path,
    target_electrode_deformed_path=target_electrode_deformed_path,
    target_pts=target_pts, 
    islabel=True  # mark center of electrode and tip of electrode
)

---

## Summary

This notebook demonstrated a complete pipeline for rodent brain atlas registration and electrode deformation analysis:

### Registration Pipeline:
1. **Rigid Registration** - Initial center alignment
2. **Affine Registration** - Linear transformation refinement  
3. **Non-linear Registration** - Deformable registration for precise anatomical matching
4. **Quality Assessment** - Jacobian determinant analysis

### Electrode Deformation Modeling:
- Specialized deformation field modeling around electrode tracts
- Sequential processing for multiple electrodes
- Tissue displacement correction for improved registration accuracy

### Key Outputs:
- Registered atlas images at each transformation stage
- Anatomical labels aligned to subject space
- Quality metrics (Jacobian determinants)
- Electrode deformation-corrected registrations

### Applications:
- Accurate anatomical labeling of rodent brain regions
- Quantification of tissue deformation due to electrode implantation  
- Quality control for registration accuracy
- Research in neuroscience and medical imaging

---

**Note:** File paths in this notebook are configured for a specific dataset (R57). Update the file paths in section 2 to match your data organization before running the pipeline.